In [83]:
import numpy as np
np.random.seed(220)

class EventList:
    def __init__(self):
        self.time = []
        self.entity = []
        self.event = []
    
    def __str__(self):
        res = str(self.time[0])+ "\n"
        res += "#: Time  \t Customer \t\t Event " +"\n"
        for i in range(len(self.time)):
            res+=str(i)+":"+" {:.3f}\t {:} \t{:}".format(float(self.time[i]),self.entity[i],self.event[i])+"\n"
        return res
    
    def __repr__(self):
        res = ""
        for i in range(len(self.time)):
            res+=str(i)+":"+" {:.3f}\t {:}\t {:}".format(float(self.time[i]),self.entity[i],self.event[i])+"\n"
        return res
    
    def addEvent(self,time,entity,event):
        """
        Adds an event to the EventList object. The data stored is the time the event occurred, the entity that did something?, and the event that happened (eg time 0, server, served a customer)
        """
        curr = time
        self.time.append(time)
        self.time.sort()
        idx = self.time.index(curr)
        self.entity.insert(idx,entity)
        self.event.insert(idx,event)
        
    def nextEvent(self):
        """
        Modifies the event list, such that the first event is deleted and then returns the event after it. (eg if event list is [(0, server, served customer A), (2.344, server, served customer B)], the first element is deleted and the second element is returned)
        """
        time = self.time[0]
        entity = self.entity[0]
        event = self.event[0]
        del (self.time[0])
        del (self.entity[0])
        del (self.event[0])
        return time,entity,event
    
class Queue:
    '''
    Basic FIFO queue
    '''
    def __init__(self):
        self.items = []
        
    def __repr__(self):
        return str(self.items)
        
    def __str__(self):
        return str(self.items)
    
    def __len__(self):
        return len(self.items)
    
    def enqueue(self, item):
        '''
        Adds item to queue.
        '''
        self.items.append(item)
        
    def dequeue(self):
        '''
        Returns first item in queue
        '''
        return self.items.pop(0)
    
    def isEmpty(self):
        '''
        Boolean of whether or not queue is empty
        '''
        return len(self.items) == 0
    
    def nonEmpty(self):
        '''
        Boolean of whether or not queue is non empty
        '''
        return not self.isEmpty()
    
class Customer:
    '''
    Customer object, only attribute is arrival time
    '''
    def __init__(self, arriv_time, number):
        self.Atime = arriv_time
        self.number = number
        self.Dtime = -1
        
    
    def __repr__(self):
        return f"Customer({self.time:.3f})"
    
    def __str__(self):
        return f"{self.number}:{self.Atime:.3f},{self.Dtime:.3f}"
    
    def getTime(self):
        '''
        Returns arrival time
        '''
        return self.Atime

class Server:
    '''
    Server object
    '''
    def __init__(self, server_number, service_rate):
        self.busy = False
        self.number = server_number
        self.rate = service_rate
        self.customer = None
            
    
    def __repr__(self):
        if self.customer != None:
            return f"Server {self.number}: Customer=> {self.customer.number}:{self.customer.Atime},{self.customer.Dtime};Busy=>{self.busy}; ServiceRate=>{self.rate}"
        else:
            return f"Server {self.number}: Customer=> None;Busy=>False; ServiceRate=>{self.rate}"
    
    def isBusy(self):
        '''
        Boolean of whether or not server is busy
        '''
        return self.busy
    
    def notBusy(self):
        '''
        Boolean of whether or not server is free
        '''
        return not self.isBusy()
    
    def makeBusy(self):
        '''
        Changes server to busy
        '''
        self.busy = True
        
    def freeUp(self):
        '''
        Makes server not busy
        '''
        self.busy = False
        self.customer = None
    
    def serve(self, cust, arrive_time):
        '''
        Used to schedule a customer's departure
        Inputs:
            cust - customer
            arrive_time - time customer gets to server
            serve_rate - how fast server serves
        Outputs:
            A scheduled time based on arrival time, server rate, and the
            exponential distribution
        '''
        self.makeBusy()
        self.customer = cust
        self.customer_number = cust.number
        time_to_serve = np.random.exponential(self.rate,1)[0]
        cust.Dtime = arrive_time + time_to_serve
        return cust.Dtime
        

def generatePop(arrival_rate, max_time):
    '''
    Helper function that creates a population of shoppers to funnel into the 
    "shop"
    Inputs:
        arrival_rate - rate at which customers arrive to shop
        max_time - amount of time after 0 that they will keep coming to the shop
    Returns:
        Queue containing customers in order of their arrival time at the shop
    '''
    t = 0
    cust = 1
    
    population = Queue()
    while t < max_time:
        arriv = np.random.exponential(arrival_rate,1)[0]+t
        c = Customer(arriv,cust)
        population.enqueue(c)
        t = arriv
        cust+=1
    return population


In [107]:
def simulateQueue(arrival_rate, dept_rate, max_time, num_servers):
    '''
    Simulates a shop by taking people from the shopping population, assinging
    them to the server if the server is free, or assinging them to wait in the
    queue otherwise.
    Inputs:
        arrival_rate - rate at which customers arrive to shop
        dept_rate - a list of service rates
        max_time - amount of time after 0 that they will keep coming to the shop
    Outputs:
        If rnt == False: none, prints the event list
        If rtn == True: average wait time in queue
    '''
    population = generatePop(arrival_rate, max_time)
    q = Queue()
    server_list = []
    for x in range(num_servers):
        server_list.append(Server(x+1, dept_rate[x]))
    e = EventList()
    t = 0
    while population.nonEmpty() or q.nonEmpty():
        available_server = True #assume there is at least one server available
        for server in server_list:
            if server.isBusy():
                if server.customer.Dtime<t:
                    server.freeUp()
        if q.isEmpty():
            c = population.dequeue()
            t = c.getTime()
            #loop through the server list and see if any are available to serve the customer
            for server in server_list:
                if server.notBusy():
                    break
                else:
                    available_server = False
            if available_server == True:
                for i, server in enumerate(server_list):
                    if server.notBusy():
                        e.addEvent(t, c, "Arrival")
                        sTime1 = server.serve(c, t)
                        server_ind = i
                        e.addEvent(sTime1, c, "Departure")
                        break
            #if none of the servers were available add the customer to the queue
            else:
                q.enqueue(c)
        elif population.nonEmpty() and q.nonEmpty():
            c = population.dequeue()
            q.enqueue(c)
            t = c.getTime()
            for server in server_list:
                if server.notBusy():
                    break
                else:
                    available_server = False
            if available_server == True:
                for i, server in enumerate(server_list):
                    if server.notBusy():
                        e.addEvent(t, c, "Arrival")
                        sTime2 = server.serve(c, t)
                        e.addEvent(sTime2, c, "Departure")
                        break
        else:
            c = q.dequeue()
            e.addEvent(c.getTime(), c, "Arrival")
            for i, server in enumerate(server_list):
                if server.notBusy():
                    sTime3 = server.serve(c, t)
                    e.addEvent(sTime3, c, "Departure")
                    t = sTime3
                    break
    return e


In [108]:
sim_info = simulateQueue(5, [10,12,15], 100, 1)
print(sim_info)

2.7596528798267057
#: Time  	 Customer 		 Event 
0: 2.760	 1:2.760,3.606 	Arrival
1: 3.088	 2:3.088,119.115 	Arrival
2: 3.606	 1:2.760,3.606 	Departure
3: 14.449	 3:14.449,-1.000 	Arrival
4: 17.564	 4:17.564,24.996 	Arrival
5: 17.564	 4:17.564,24.996 	Arrival
6: 20.505	 5:20.505,-1.000 	Arrival
7: 21.108	 6:21.108,-1.000 	Arrival
8: 24.996	 4:17.564,24.996 	Departure
9: 25.691	 7:25.691,-1.000 	Arrival
10: 30.451	 8:30.451,33.612 	Arrival
11: 30.451	 8:30.451,33.612 	Arrival
12: 32.064	 9:32.064,-1.000 	Arrival
13: 33.612	 8:30.451,33.612 	Departure
14: 35.631	 10:35.631,-1.000 	Arrival
15: 36.929	 11:36.929,55.794 	Arrival
16: 36.929	 11:36.929,55.794 	Arrival
17: 42.799	 12:42.799,-1.000 	Arrival
18: 50.317	 13:50.317,-1.000 	Arrival
19: 55.794	 11:36.929,55.794 	Departure
20: 56.169	 14:56.169,-1.000 	Arrival
21: 61.011	 15:61.011,71.618 	Arrival
22: 61.011	 15:61.011,71.618 	Arrival
23: 61.715	 16:61.715,-1.000 	Arrival
24: 65.346	 17:65.346,-1.000 	Arrival
25: 71.618	 15:61.011,71

In [0]:
def simulateQueue(arrival_rate, dept_rate, max_time, num_servers):
    '''
    Simulates a shop by taking people from the shopping population, assinging
    them to the server if the server is free, or assinging them to wait in the
    queue otherwise.
    Inputs:
        arrival_rate - rate at which customers arrive to shop
        dept_rate - a list of service rates
        max_time - amount of time after 0 that they will keep coming to the shop
    Outputs:
        If rnt == False: none, prints the event list
        If rtn == True: average wait time in queue
    '''
    population = generatePop(arrival_rate, max_time)
    q = Queue()
    server_list = []
    for x in range(num_servers):
        server_list.append(Server(x+1, dept_rate[x]))
    e = EventList()
    t = 0
    while population.nonEmpty() or q.nonEmpty():
        available_server = True #assume there is at least one server available
        if q.isEmpty():
            c = population.dequeue()
            t = c.getTime()
            #loop through the server list and see if any are available to serve the customer
            for server in server_list:
                if server.notBusy():
                    break
                else:
                    available_server = False
            if available_server == True:
                for i, server in enumerate(server_list):
                    if server.notBusy():
                        e.addEvent(t, c, "Arrival")
                        sTime1 = server.serve(c, t)
                        server_ind = i
                        e.addEvent(sTime1, c, "Departure")
                        break
            #if none of the servers were available add the customer to the queue
            else:
                q.enqueue(c)
                if t >= sTime1:
                    server_list[i].freeUp()
        elif population.nonEmpty() and q.nonEmpty():
            c = population.dequeue()
            q.enqueue(c)
            t = c.getTime()
            for server in server_list:
                if server.notBusy():
                    break
                else:
                    available_server = False
            if available_server == True:
                for i, server in enumerate(server_list):
                    if server.notBusy():
                        e.addEvent(t, c, "Arrival")
                        sTime2 = server.serve(c, t)
                        e.addEvent(sTime2, c, "Departure")
                        break
            else:
                if t >= sTime1:
                    server_list[i].freeUp()
        else:
            c = q.dequeue()
            e.addEvent(c.getTime(), c, "Arrival")
            for i, server in enumerate(server_list):
                if server.notBusy():
                    sTime3 = server.serve(c, t)
                    e.addEvent(sTime3, c, "Departure")
                    break
            t = sTime3
    return e